<h1>IBM Data Science Capstone Project</h1>
<h3>This project is recomendation for an client who wants to open a Gym in Toronto, York area</h3>

In [1]:
#import libraries 

#pip install geopy
#pip install folium

import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [2]:
#Scrape Wikipedia page to obtain the Tornto data and transform the data into a pandas dataframe
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
print(df.shape)
df.head()

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
#Ignore cells with a Borough that have Not assigned value.
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
#Check if we have Neighbourhood = Not assigned
df[df['Neighbourhood']== 'Not assigned']


,Postal Code,Borough,Neighbourhood


In [4]:
#check the size of Postal code dataframe
print('The shape of the dataset is:',df.shape)

The shape of the dataset is: (103, 3)


In [5]:
#Get the latitude and the longitude coordinates of each neighborhood.
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
#Merge both dataframes on Postal Code column
df_neighborhoods_coordinates = pd.merge(df, df_coordinates, on='Postal Code')
df_neighborhoods_coordinates.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [7]:
#Rename column Neighbourhood and check dataframe size
df2=df_neighborhoods_coordinates.rename({'Neighbourhood': 'Neighborhood'}, axis=1) 
#check the size of Postal code dataframe
print('The shape of the dataset is:',df2.shape)

The shape of the dataset is: (103, 5)


In [8]:
#Group it by Bourough and count Neighborhood
df2.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [9]:
#Simplify, slice the original dataframe and create a new dataframe, include only York area
df_york = df2[df2['Borough'].str.contains('York')]
df_york.reset_index(inplace=True)
df_york.drop('index', axis=1, inplace=True)
df_york.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


In [10]:
#Group it by Bourough and count Neighborhood
df_york.groupby('Borough').count()['Neighborhood']

Borough
East York      5
North York    24
York           5
Name: Neighborhood, dtype: int64

In [11]:
#Get the geographical coordinates of Torornto
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [12]:
#Create a map of Toronto with York Neighborhood
map_york = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_york['Latitude'], df_york['Longitude'], df_york['Borough'], df_york['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
    
map_york

<h1>Utilizing the Foursquare API to explore the neighborhoods and segment them.</h1>

In [13]:
# The code was removed by Watson Studio for sharing.

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
#Using function above get venues in york dataset
york_venues = getNearbyVenues(names=df_york['Neighborhood'],
                                latitudes=df_york['Latitude'],
                                longitudes=df_york['Longitude'])

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Parkview Hill, Woodbine Gardens
Glencairn
Don Mills
Woodbine Heights
Humewood-Cedarvale
Caledonia-Fairbanks
Leaside
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Humberlea, Emery
Willowdale, Willowdale East
Downsview
Runnymede, The Junction North
Weston
York Mills West
Willowdale, Willowdale West


In [16]:
#Check size
york_venues.shape

(336, 7)

In [17]:
york_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [18]:
#Check number of Venue Categories
len(york_venues['Venue Category'].unique()) 

120

In [19]:
#Check list of Venue Categories
york_venues['Venue Category'].unique()[:100]

array(['Park', 'Food & Drink Shop', 'Hockey Arena',
       'Portuguese Restaurant', 'Coffee Shop', 'Intersection',
       'Pizza Place', 'Boutique', 'Furniture / Home Store',
       'Vietnamese Restaurant', 'Clothing Store', 'Accessories Store',
       'Event Space', "Women's Store", 'Gift Shop',
       'Caribbean Restaurant', 'Gym', 'Café', 'Japanese Restaurant',
       'Gastropub', 'Gym / Fitness Center', 'Pharmacy', 'Bank',
       'Breakfast Spot', 'Pet Store', 'Athletics & Sports', 'Pub',
       'Italian Restaurant', 'Discount Store', 'Restaurant',
       'Art Gallery', 'Bike Shop', 'Sporting Goods Shop', 'Beer Store',
       'Supermarket', 'Dim Sum Restaurant', 'Asian Restaurant',
       'Sandwich Place', 'Chinese Restaurant', 'Skating Rink',
       'Curling Ice', 'Dance Studio', 'Field', 'Trail', 'Dog Run',
       'Tennis Court', 'Pool', 'Sports Bar', 'Fish & Chips Shop',
       'Liquor Store', 'Grocery Store', 'Smoothie Shop',
       'Sushi Restaurant', 'Burger Joint', 'Shopping

In [20]:
# check if the results contain "Gym"
"Gym" in york_venues['Venue Category'].unique()

True

<h1>Analyze Each Neighborhood</h1>

In [21]:
# one hot encoding
to_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(336, 121)


,Neighborhoods,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
#Group by Neighborhood and calculate mean for each Venue Category
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(29, 121)


,Neighborhoods,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.0000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.045455,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,Caledonia-Fairbanks,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.250000,0.000000
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.2,0.000000,0.000000,0.000000,0.000000,0.000000
5,Don Mills,0.000000,0.0000,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,Downsview,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,"East Toronto, Broadview North (Old East York)",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Fairview, Henry Farm, Oriole",0.000000,0.0000,0.014286,0.000000,0.000000,0.014286,0.000000,0.000000,0.028571,...,0.000000,0.014286,0.014286,0.0,0.0,0.014286,0.000000,0.000000,0.042857,0.000000
9,Glencairn,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [23]:
#How many categories = Gym
len(to_grouped[to_grouped["Gym"] > 0])

2

In [24]:
#Create a new dataframe to find GYms only
to_gym = to_grouped[["Neighborhoods","Gym"]]

In [25]:
to_gym.head(10)

,Neighborhoods,Gym
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000
1,Bayview Village,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000
3,Caledonia-Fairbanks,0.000000
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.000000
5,Don Mills,0.130435
6,Downsview,0.000000
7,"East Toronto, Broadview North (Old East York)",0.000000
8,"Fairview, Henry Farm, Oriole",0.000000
9,Glencairn,0.000000


<h1>Cluster Neighborhoods</h1>

In [26]:
#Run k-means to cluster the neighborhoods in York into 3 clusters.

# set number of clusters
toclusters = 3

to_clustering = to_gym.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int32)

In [27]:
#Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = to_gym.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head(10)

,Neighborhood,Gym,Cluster Labels
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0
1,Bayview Village,0.000000,0
2,"Bedford Park, Lawrence Manor East",0.000000,0
3,Caledonia-Fairbanks,0.000000,0
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.000000,0
5,Don Mills,0.130435,1
6,Downsview,0.000000,0
7,"East Toronto, Broadview North (Old East York)",0.000000,0
8,"Fairview, Henry Farm, Oriole",0.000000,0
9,Glencairn,0.000000,0


In [28]:
# merge york_grouped with york_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(york_venues.set_index("Neighborhood"), on="Neighborhood")
#sort it by Cluster Labels
to_merged.sort_values(["Cluster Labels"], inplace=True)

print(to_merged.shape)
to_merged

(336, 9)


,Neighborhood,Gym,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0,43.754328,-79.442259,Wolfie's Deli,43.754875,-79.442438,Deli / Bodega
15,Leaside,0.000000,0,43.709060,-79.363452,Tim Hortons,43.705629,-79.361028,Coffee Shop
15,Leaside,0.000000,0,43.709060,-79.363452,South St. Burger,43.710510,-79.361584,Burger Joint
15,Leaside,0.000000,0,43.709060,-79.363452,Kintako Japanese Restaurant,43.711597,-79.363962,Sushi Restaurant
15,Leaside,0.000000,0,43.709060,-79.363452,Booster Juice,43.706173,-79.360652,Smoothie Shop
...,...,...,...,...,...,...,...,...,...
21,Thorncliffe Park,0.041667,2,43.705369,-79.349372,Bikram Yoga East York,43.705450,-79.351448,Yoga Studio
21,Thorncliffe Park,0.041667,2,43.705369,-79.349372,Fit4Less,43.705689,-79.346018,Gym
21,Thorncliffe Park,0.041667,2,43.705369,-79.349372,Iqbal Kebab & Sweet Centre,43.705923,-79.351521,Indian Restaurant
21,Thorncliffe Park,0.041667,2,43.705369,-79.349372,Hakka Garden,43.704578,-79.349770,Indian Restaurant


In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)

cluster_colours = ['red', 'green','blue']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=cluster_colours[cluster-1],
        fill_color=cluster_colours[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1>Examine Clusters</h1>

In [30]:
#Cluster 0
cluster0 = to_merged.loc[to_merged['Cluster Labels'] == 0]
cluster0[cluster0['Venue Category'].str.contains('Gym')]


,Neighborhood,Gym,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
18,"Parkview Hill, Woodbine Gardens",0.0,0,43.706397,-79.309937,East York Gymnastics,43.710654,-79.309279,Gym / Fitness Center
6,Downsview,0.0,0,43.761631,-79.520999,Planet Fitness,43.757538,-79.519610,Gym / Fitness Center


In [31]:
#Cluster 1
cluster1 = to_merged.loc[to_merged['Cluster Labels'] == 1]
cluster1[cluster1['Venue Category'].str.contains('Gym')]

,Neighborhood,Gym,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,Don Mills,0.130435,1,43.725900,-79.340923,Fitness Connection,43.727473,-79.341707,Gym
5,Don Mills,0.130435,1,43.725900,-79.340923,GoodLife Fitness North York Don Mills and Egli...,43.722704,-79.337508,Gym
5,Don Mills,0.130435,1,43.745906,-79.352188,LA Fitness,43.747665,-79.347077,Gym


In [32]:
#Cluster 2
cluster2 = to_merged.loc[to_merged['Cluster Labels'] == 2]
cluster2[cluster2['Venue Category'].str.contains('Gym')]

,Neighborhood,Gym,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
21,Thorncliffe Park,0.041667,2,43.705369,-79.349372,Cross Fit Quantum,43.706883,-79.350819,Gym / Fitness Center
21,Thorncliffe Park,0.041667,2,43.705369,-79.349372,Fit4Less,43.705689,-79.346018,Gym


<h3>After examining York area data it looks like Parkview Hill, Woodbine Gardens neighborhood might be a good location to open a gym. Most of the gyms are located in Don Mills and less gyms in Thorncliffe Park</h3>